# 04.3- Python + LLMs

<div class="alert alert-warning">
    <b>You will need to setup an OpenAI key to fully use this notebook.</b>
    <b>Instructions will be sent via email.</b>
</div>

Using an LLM on a prompt? By now you have most likely have done that.  
Copy pasting inputs and outputs and whatnot. Like if it is still 2023.  
In this notebook we'll introduce how to get a working python interaction with an LLM that you can later include in your projects.  
First, we need some brief definitions:

## Intro to Large Language Models (in a programatic sense)

### LLMs: Large Language Models
A Large Language Model is a type of artificial intelligence model trained on vast amounts of text data to understand and generate human-like language. Think about our Shakespeare example, but on an a much much larger scale. These models are based on deep learning techniques, specifically neural networks, and are capable of performing various natural language processing tasks.

### Generative Models
While LLMs handle language, other models may generate art, recognize sound (Speech-To-Text), create videos, etc.. For coding purposes, one usually uses an LLM optimized for code, for example. To generate images you might use DALL-E and so on.

### Agents
Agents are software systems that have a [model at its core](https://developer.nvidia.com/blog/introduction-to-llm-agents/). The agents add memory to the model, making the model "aware" of the history of a chat you are having. The model can be any kind of generative model (images, text, etc.). An agent might even use several AI models under the hood, making it multi-task.

### Langchain
[Langchain](https://en.wikipedia.org/wiki/LangChain) is a software framework designed to make your interaction with AI Models and Agents much easier. It is still in its early stages, so we will use it with caution.

### In this course
We are going to focus on LLMs suited for coding and data analysis. Please make sure you have the secret key configured for OpenAI.

In [1]:
# Uncomment and run, or copy paste to terminal and run
!pip install langchain
!pip install -U langchain-openai
#You might need to restart the kernel after installing

  Using cached langchain-0.1.8-py3-none-any.whl.metadata (13 kB)
  Using cached SQLAlchemy-2.0.27-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.9.3-cp311-cp311-win_amd64.whl.metadata (7.6 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.21-py3-none-any.whl.metadata (8.1 kB)
  Using cached langchain_core-0.1.24-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.1.3-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pydantic-2.6.1-py3-none-any.whl.metadata (83 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp311-cp311-win_amd64.whl.metadata (4.3 kB)
  Using cached

In [2]:
# We are going to use a model from the OpenAI family
from langchain_openai import OpenAI

### Using models with Langchain

The `langchain` framework has dedicated modules for specific models. For example, if a model ws developed by OpenAI, there is a dedicated `langchain-openai` set of tools.  
This is all still pretty new, as you can see by the version numbers:

In [3]:
import langchain
langchain.__version__

'0.1.8'

In this course we are going to use OpenAI models for three reasons:
1. Maturity of the integration between OpeanAI and Langchain
2. Quality/Price of the standard model used: gpt-3.5-turbo is cheap and provides adequate results for coding/data analysis.
3. Results are more in line with ChatGPT, which is what students may be more familiar with (same family of models)

Using the model requires payment, but it is the only paid component of this course. There are open source models that you can deploy on your machine if you are developing a start-up, for example.

More resources:
* https://github.com/Hannibal046/Awesome-LLM
* https://github.com/langchain-ai/langchain/tree/master/cookbook

---
### Let's use a model

If there is one thing you should take from this course is that Documentation exists for a reason.  
Take a look at the OpenAI documentation for models: https://platform.openai.com/docs/models

#### Written by chatgpt: (in this course we will focus only on the first four)
When using a large language model like GPT-3, there are several key parameters that you can adjust to customize the behavior of the model for your specific task or application. Here are some of the main parameters:
* Model Size: The size of the model refers to the number of parameters it has. Larger models typically have more capacity to understand and generate complex language, but they also require more computational resources.
* Context Length: This parameter determines the amount of context the model considers when generating responses. A longer context allows the model to better understand the input, but it also increases computational requirements.
* Temperature: Temperature is a parameter that controls the randomness of the model's output. Higher temperature values (e.g., 0.8) lead to more creative and diverse responses, while lower values (e.g., 0.2) make the output more deterministic and focused.
* Max Tokens: This parameter limits the length of the generated output. It can be useful to prevent overly long responses.
* Top-k and Top-p/Nucleus Sampling: These parameters control the diversity of the generated output. Top-k sampling selects the top k most likely next tokens, while top-p (nucleus) sampling selects tokens until the cumulative probability exceeds a certain threshold p.
* Frequency Penalties: You can apply penalties or bonuses based on the frequency of tokens in the output. This can be used to encourage or discourage the use of specific words or phrases.
* Prompt Engineering: Crafting a well-defined and clear prompt can significantly impact the quality and relevance of the model's responses. Experimenting with different prompts can help achieve the desired output.
* Fine-tuning: If you have specific requirements for your task, you can fine-tune the model on a custom dataset. Fine-tuning allows the model to learn task-specific patterns and improve performance.
* Resource Allocation: Depending on the hardware you are using, you may need to adjust parameters to fit within the available resources. Larger models and longer contexts require more memory and processing power.
* API Usage: When using a language model through an API, there might be additional parameters or constraints specific to the API. Understanding the API documentation is crucial for effective utilization.

It's essential to experiment with these parameters to find the right balance for your specific use case. Keep in mind that tweaking parameters may require some trial and error to achieve the desired results.

---

Since we are going to do introductory level requests, and are looking mostly for coding solutions, we will keep the temperature low.

In [4]:
llm = ChatOpenAI(temperature=0.1) #Check shift+Tab

In [5]:
#Play around with the autocomplete. Check what methods and attributes our instance "llm" has.
llm.model_name

'gpt-3.5-turbo'

#### The most basic interaction

In [6]:
llm.invoke('Here is a fun fact about Optimus Prime:')

AIMessage(content='Optimus Prime was originally going to be named "Orion Pax" in the Transformers series, but the name was changed to Optimus Prime before the character was introduced.')

In [7]:
result = llm.invoke('Which music album is considered the best of all time by critics?')

In [8]:
result.content

'There is no definitive answer to this question as opinions on the best music album of all time vary among critics. However, some albums that are often cited as contenders for the title of best album of all time include:\n\n- The Beatles - "Sgt. Pepper\'s Lonely Hearts Club Band"\n- Pink Floyd - "The Dark Side of the Moon"\n- Bob Dylan - "Highway 61 Revisited"\n- The Beach Boys - "Pet Sounds"\n- Michael Jackson - "Thriller"\n\nUltimately, the best music album of all time is subjective and depends on individual taste and preferences.'

#### For better printing on a Notebook

In [9]:
from IPython.display import Markdown

In [10]:
Markdown(result.content)

There is no definitive answer to this question as opinions on the best music album of all time vary among critics. However, some albums that are often cited as contenders for the title of best album of all time include:

- The Beatles - "Sgt. Pepper's Lonely Hearts Club Band"
- Pink Floyd - "The Dark Side of the Moon"
- Bob Dylan - "Highway 61 Revisited"
- The Beach Boys - "Pet Sounds"
- Michael Jackson - "Thriller"

Ultimately, the best music album of all time is subjective and depends on individual taste and preferences.

In [11]:
result = llm.invoke('Please remove albums released later than 1990.')

In [12]:
Markdown(result.content)

I'm sorry, but I cannot provide a list of albums released before 1990 as I am unable to access specific information about album release dates. If you have any other requests or questions, feel free to ask.

The LLM has no recollection of your previous message. We are doing "Zero-Shot calls" to the model, meaning there is no notion of state. In other words, the instance you started has no memory of past prompts.